In [31]:
from datetime import date, datetime, timedelta

#### liste des fonctions à créer
- Échange
- Ballotage/Libérer joueur
- Commutations
    - Blessures
- Repêchages

In [3]:
#créer la table DRAFT en SQL avec les choix au repêchage
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
#choix = pd.read_csv("C:\dev\pool-de-hockey\src\data\listeChoix.csv", sep=";")
#choix.to_sql('DRAFT', conn, if_exists='replace', index = False)
def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

In [3]:
#DO NOT EXECUTE créer une fausse table ALIGNEMENTS POUR FAIRE LES TESTS
#alignements = pd.read_csv("alignements.csv", sep=";")
#alignements.to_sql('copALIGN', conn, if_exists='replace', index = False)

In [148]:
# créer la fonction qui effectue l'échange pour un élément JOUEUR de l'échange

# la fonction doit faire deux choses:
        # 1. DONE terminer l'ancienne période du joueur en changeant simplement dateFin
        # 2. DONE ajouter la nouvelle période du joueur dans sa nouvelle équipe

#requête qui va chercher la ligne actuelle d'un joueur dans ALIGNEMENTS
def joueurLive(id):
    requete = "SELECT * FROM copALIGN WHERE idNHL == " + id + " AND dateFin == '2100-01-01'"
    df = pd.read_sql(requete, conn)
    return df

#fonction qui termine la période actuelle des joueurs de l'échange
def finirPeriode(joueurs, dat):
    dat = datetime.strptime(dat, "%Y-%m-%d")
    dat = dat - timedelta(days=1)
    dat = dat.strftime("%Y-%m-%d")
    dat = str(dat)
    for i in joueurs:
        str_i = str(i)
        df = joueurLive(str_i)
        idAlign = df.iloc[0]["idAlignement"]
        idAlign = str(idAlign)
        req = "UPDATE copALIGN SET dateFin = '" + dat + "' WHERE idAlignement == " + idAlign
        execSQL(req, conn, write=True)


#écrire la nouvelle ligne du joueur
def nouvellePeriode(id, dat):
    strId = str(id)
        # idAlignement : max de idAlignement + 1
    reqIdAlign = "SELECT MAX(idAlignement) FROM copALIGN"
    W_idAlign = pd.read_sql(reqIdAlign, conn)
    W_idAlign = W_idAlign.iloc[0]["MAX(idAlignement)"] + 1
        # idPooler : aller le chercher dans testTrade.csv avec idNHL
    is_idNHL = trade["idNHL"] == id
    joueurTrade = trade[is_idNHL]
    W_idPooler = joueurTrade.iloc[0]["equipeRecoit"]
        # prenomJoueur : aller le chercher dans JOUEURS avec idNHL
    reqPrenom = "SELECT prenomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_prenom = pd.read_sql(reqPrenom, conn)
    W_prenom = W_prenom.iloc[0]["prenomJoueur"]
        # nomJoueur : aller le chercher dans JOUEURS avec idNHL
    reqNom = "SELECT nomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_nom = pd.read_sql(reqNom, conn)
    W_nom = W_nom.iloc[0]["nomJoueur"]
        # idNHL : paramètre de la fonction
        # statutJoueur : aller le chercher dans joueurLive(id)
    df = joueurLive(strId)
    W_statut = df.iloc[0]["statutJoueur"]
        # dateDebut : paramètre de la fonction
        # dateFin : '2100-01-01'
    W_dateFin = "2100-01-01"

    d = {"idAlignement" : [W_idAlign], "idPooler" : [W_idPooler], "prenomJoueur" : [W_prenom], "nomJoueur" : [W_nom], "idNHL" : [id], "statutJoueur" : [W_statut], "dateDebut" : [dat], "dateFin" : [W_dateFin], "pjActuels" : [0], "butsActuels" : [0], "assistActuels" : [0], "pointsActuels": [0]}
    periode = pd.DataFrame(data=d)
    periode.to_sql("copALIGN", conn, if_exists="append", index=False)
    ok = W_prenom + " " + W_nom
    return ok

#fonction get noms de joueur
def getNom(id):
    strId = str(id)
    reqPrenom = "SELECT prenomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_prenom = pd.read_sql(reqPrenom, conn)
    W_prenom = W_prenom.iloc[0]["prenomJoueur"]
    reqNom = "SELECT nomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_nom = pd.read_sql(reqNom, conn)
    W_nom = W_nom.iloc[0]["nomJoueur"]
    return W_prenom + " " + W_nom

#ajouter l'échange au registre
def ajouterRegistre(dat, condition):
    #générer le idTrade
    req = "SELECT MAX(idTrade) FROM ÉCHANGES"
    m = pd.read_sql(req, conn)
    W_idTrade = m.iloc[0]["MAX(idTrade)"] + 1
    #générer EquipeA
    equipes = trade["equipeRecoit"].unique()
    equipeA = equipes[0]
    #générer JoueursA
    is_equA = trade["equipeRecoit"] == equipeA
    joueursA = trade[is_equA]
    joueursA = joueursA["idNHL"]
    a = []
    for i in joueursA:
        nom = getNom(i)
        a.__iadd__([nom])
    str_a = ', '.join([str(elem) for elem in a])
    #générer EquipeB
    equipeB = equipes[1]
    #générer JoueursB
    is_equB = trade["equipeRecoit"] == equipeB
    joueursB = trade[is_equB]
    joueursB = joueursB["idNHL"]
    b = []
    for i in joueursB:
        nom = getNom(i)
        b.__iadd__([nom])
    str_b = ', '.join([str(elem) for elem in b])
    d = {"idTrade" : [W_idTrade], "DateTrade" : [dat], "EquipeA" : [equipeA], "JoueursA" : [str_a], "EquipeB" : [equipeB], "JoueursB" : [str_b], "Condition" : [condition]}
    echange = pd.DataFrame(data=d)
    echange.to_sql("ÉCHANGES", conn, if_exists="append", index=False)
    ok = "ok"
    return echange

def echange(joueurs, dat, condition="vide"):
    for i in joueurs:
        nouvellePeriode(i, dat)
    finirPeriode(joueurs, dat)
    done = "done"
    ajouterRegistre(dat, condition)
    return done

In [149]:
trade = pd.read_csv("testTrade.csv", sep=";")
ids = trade["idNHL"]
dat = "2021-02-19"

echange(ids, dat)

'done'